In [57]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import numpy as np

# Dataset: Intel Image Classification
# https://www.kaggle.com/puneet6060/intel-image-classification/version/2
# 25k imagini 150 x 150, 6 clase

In [58]:
#Crearea dataset-urilor

batch_size = 256
image_size = (150, 150)


train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../input/intel-image-classification/seg_train/seg_train",
    seed=2022,
    image_size=image_size,
    batch_size=batch_size,
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../input/intel-image-classification/seg_test/seg_test",
    seed=2022,
    image_size=image_size,
    batch_size=batch_size,
)

In [59]:
# Afisarea claselor
nume_clase = train_dataset.class_names
print(nume_clase)

In [114]:
# Afisarea a 9 imagini din setul de antrenare
plt.figure(figsize=(10, 10))
for images, labels in train_dataset:
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(nume_clase[labels[i]])
    plt.axis("off")

In [115]:
# Shape-ul dataset-ului de antrenare
for image_batch, labels_batch in train_dataset:
    print(image_batch.shape)
    break

In [66]:
train_dataset = train_dataset.prefetch(buffer_size = 256)
test_dataset = test_dataset.prefetch(buffer_size = 256)

In [55]:
# Model 1 ~73% acuratete pe dataset-ul de validare
# 13s 214ms/step - loss: 0.3166 - accuracy: 0.9009 - val_loss: 0.8280 - val_accuracy: 0.7300

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1. / 255),
    tf.keras.layers.Conv2D(32, (3, 3), activation = tf.nn.relu, input_shape = (150, 150, 3)), 
    tf.keras.layers.AveragePooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = tf.nn.relu),
    tf.keras.layers.AveragePooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = tf.nn.selu),
    tf.keras.layers.Dense(6, activation = tf.nn.softmax)
])

In [67]:
# Model 2 ~80% acuratete pe dataset-ul de validare

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./ 255),
  tf.keras.layers.Conv2D(32, (3, 3), activation = tf.nn.relu, input_shape = (150, 150, 3)), 
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation = tf.nn.relu),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation = tf.nn.relu),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation = tf.nn.relu),
  tf.keras.layers.Dense(6, activation = tf.nn.softmax)
])

In [69]:
#Antrenare retea

epochs = 10

callbacks = [keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model.fit(
    train_dataset, epochs=epochs, callbacks=callbacks, validation_data=test_dataset,
)

In [70]:
model.summary()

In [138]:
import os
import random 

#Aleg o poza random din folder-ul pentru predictii
cale = "../input/intel-image-classification/seg_pred/seg_pred"
lista_poze = os.listdir(cale)
poza_random = random.choice(lista_poze)
poza_random = cale + "/" + poza_random

img = keras.preprocessing.image.load_img(poza_random, target_size=image_size)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) 


#Predictiile modelului pentru imaginea incarcata
predictii = model.predict(img_array)
predictie_label = nume_clase[np.argmax(predictii)]


#Afisari
print()
print("%.2f %% building, %.2f %% forest, %.2f %% glacier, %.2f %% mountain, %.2f %% sea, %.2f %% street"
     % (100 * predictii[0][0], 100 * predictii[0][1], 100 * predictii[0][2], 100 * predictii[0][3],
        100 * predictii[0][4], 100 * predictii[0][5]))
print()


plt.figure()
plt.imshow(img)
plt.title(predictie_label)
plt.show(block=True)

In [133]:
#Aleg o poza random din folder-ul pentru predictii
cale = "../input/intel-image-classification/seg_pred/seg_pred"
lista_poze = os.listdir(cale)
poze_random = random.choices(lista_poze,k = 9)

plt.figure(figsize=(10, 10))

for i in range(9):
    poze_random[i] = cale + "/" + poze_random[i]

    img = keras.preprocessing.image.load_img(poze_random[i], target_size=image_size)

    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) 


    #Predictiile modelului pentru imaginea incarcata
    predictii = model.predict(img_array)
    predictie_label = nume_clase[np.argmax(predictii)]


    #Afisari
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.title(predictie_label)
    plt.axis("off")